In [ ]:
#!pip install PdfReader
#!pip install python-docx
#!pip install PyPDF2
#!pip install aspose-words

In [24]:
from __future__ import annotations
import os
import PyPDF2
from PyPDF2 import PdfReader
import docx
import pandas as pd
import numpy as np
import json
import aspose.words as aw
import traceback
import re


print('docx version:', docx.__version__)
print('PyPDF2 version:',PyPDF2.__version__)

docx version: 1.1.0
PyPDF2 version: 3.0.1


In [8]:
import gdown

url = 'https://drive.google.com/file/d/1_E3GLMkbboLZcUPVfQHw99RWU11znbAq/view?usp=share_link'
file_name = 'data'
gdown.download(url, file_name, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1_E3GLMkbboLZcUPVfQHw99RWU11znbAq
To: /Users/romanisaev/VS code projects/html_parser/data
100%|██████████| 9.68M/9.68M [00:03<00:00, 2.70MB/s]


'data'

In [10]:
!unzip 'data' -d 'data_dir'

Archive:  data
   creating: data_dir/hacka-aka-embedika-2/
  inflating: data_dir/__MACOSX/._hacka-aka-embedika-2  
  inflating: data_dir/hacka-aka-embedika-2/classes.json  
  inflating: data_dir/__MACOSX/hacka-aka-embedika-2/._classes.json  
   creating: data_dir/hacka-aka-embedika-2/docs/
  inflating: data_dir/__MACOSX/hacka-aka-embedika-2/._docs  
  inflating: data_dir/hacka-aka-embedika-2/docs/02682d726b725f95b9ee85f751c043d0.doc  
  inflating: data_dir/__MACOSX/hacka-aka-embedika-2/docs/._02682d726b725f95b9ee85f751c043d0.doc  
  inflating: data_dir/hacka-aka-embedika-2/docs/2c81df29db63aebf495106881a52188f.doc  
  inflating: data_dir/__MACOSX/hacka-aka-embedika-2/docs/._2c81df29db63aebf495106881a52188f.doc  
  inflating: data_dir/hacka-aka-embedika-2/docs/31d24c0d4694e7c2fb19933b84414b40.doc  
  inflating: data_dir/__MACOSX/hacka-aka-embedika-2/docs/._31d24c0d4694e7c2fb19933b84414b40.doc  
  inflating: data_dir/hacka-aka-embedika-2/docs/57962abd8dbe0ce0c7056896ee4501f1.doc  
  infl

In [14]:
data_dir = 'data_dir/hacka-aka-embedika-2/docs'
json_path = 'data_dir/hacka-aka-embedika-2/classes.json'

converted_docx_dir = 'converted_docx'
filenames = os.listdir(data_dir)


In [15]:
def init_file_paths(data_dir_path: str) -> tuple[list[tuple[str,str]],
                                                 list[tuple[str,str]],
                                                 list[tuple[str,str]],
                                                 list[tuple[str,str]]]:
    '''
        Создаем списоки кортежей (полный путь, имя файла) для каждого расширения файла
    '''
    doc_file_paths = []
    docx_file_paths = []
    pdf_file_paths = []
    rtf_file_paths = []
    for filename in os.listdir(data_dir_path):
        if filename.endswith('.docx', -5):
            docx_file_paths.append((os.path.join(data_dir_path,filename), filename))
        elif filename.endswith('.pdf', -5):
            pdf_file_paths.append((os.path.join(data_dir_path,filename), filename))
        elif filename.endswith('.rtf', -5):
            rtf_file_paths.append((os.path.join(data_dir_path,filename), filename))
        elif filename.endswith('.doc', -5):
            doc_file_paths.append((os.path.join(data_dir_path,filename), filename))
    
    print(f'Количество pdf-файлов: {len(pdf_file_paths)}')
    print(f'Количество doc-файлов: {len(doc_file_paths)}')
    print(f'Количество docx-файлов: {len(docx_file_paths)}')
    print(f'Количество rtf-файлов: {len(rtf_file_paths)}')
    
    return doc_file_paths, docx_file_paths, pdf_file_paths, rtf_file_paths

doc_file_paths, docx_file_paths, pdf_file_paths, rtf_file_paths = init_file_paths(data_dir)
files_paths_to_docx = []
files_paths_to_docx += doc_file_paths
files_paths_to_docx += rtf_file_paths

Количество pdf-файлов: 13
Количество doc-файлов: 81
Количество docx-файлов: 22
Количество rtf-файлов: 4


In [16]:
def convert_doc_docx(doc_file_paths: str, converted_files_dir_path: str)->list[tuple[str,str]]:
    '''
        Конвертируем .doc файлы в .docx, так как python не открывает файлы .doc
        Возвращаемые кортежи содержат правилльный полный путь к файлу, но старое расширение в имени файла
        Новые файлы содержат в начале Evaluation Only. Created with Aspose.Words. Copyright 2003-2022 Aspose Pty Ltd.
    '''
    new_docx_paths = []
    file_extension = re.compile(r'\..*')
    for doc_file_path, doc_file_name in doc_file_paths:
        doc = aw.Document(doc_file_path)
        new_docx_path = os.path.join(converted_files_dir_path, file_extension.sub("", doc_file_name) + '.docx')
        doc.save(new_docx_path)
        new_docx_paths.append((new_docx_path, doc_file_name))
    return new_docx_paths
docx_file_paths += convert_doc_docx(files_paths_to_docx, converted_docx_dir)
print(f'Количество pdf-файлов: {len(pdf_file_paths)}')
print(f'Количество docx-файлов: {len(docx_file_paths)}')

Количество pdf-файлов: 13
Количество docx-файлов: 107


In [32]:
def read_docx_file(docx_file_path: str) -> str:
    '''
        Открываем .docx файл и, соединяя все абзацы,
        возращем полный текст документа
        или None если не удалось открыть файл
    '''
    try:
        doc = docx.Document(docx_file_path)
        all_paras = doc.paragraphs
        full_doc_text = ''
        for para in all_paras:
            full_doc_text = full_doc_text + '\n' + para.text
        return full_doc_text.replace('Evaluation Only. Created with Aspose.Words. Copyright 2003-2022 Aspose Pty Ltd.', '')
    except:
        print(f'Ошибка: Не удалось открыть файл: {docx_file_path}\n {traceback.format_exc()}')
        return None

def read_pdf_file(pdf_file_path: str) -> str:
    '''
        Открываем .pdf файл и, соединяя все страницы,
        возращем полный текст документа
        или None если не удалось открыть файл
    '''
    try:
        full_pdf_text = ''
        with open(pdf_file_path, "rb") as filehandle:
            pdf = PdfReader(filehandle)
            num_pages = len(pdf.pages)
            for num in range(num_pages):
                page = pdf.pages[num]
                full_pdf_text = full_pdf_text + '\n' + page.extract_text()
        return full_pdf_text
    except:
        print(f'Ошибка: Не удалось открыть файл: {pdf_file_path}\n {traceback.format_exc()}')
        return None

In [33]:
def get_document_text(document_file_path: str) -> str:
    re_file_extension = re.compile(r'\.\w+')

    file_extension = re_file_extension.findall(document_file_path)
    if file_extension[0] == '.docx':
        return read_docx_file(document_file_path)
    
    elif file_extension[0] == '.doc' or file_extension[0] == '.rtf':
        doc = aw.Document(document_file_path)
        new_docx_path = re_file_extension.sub('',docx_file_paths[0][0]) + '.docx'
        doc.save(new_docx_path)
        return read_docx_file(new_docx_path)

    elif file_extension[0] == '.pdf':
        return read_pdf_file(document_file_path)